In [6]:
INPUT_AUDIO_FOLDER = 'inputAudio'
INPUT_AUDIO_FILENAME = 'marine'
OUTPUT_AUDIO_FOLDER = 'outputAudio'

import librosa
import ruptures as rpt
import matplotlib.pyplot as plt
from spleeter.separator import Separator

In [ ]:


# Initialize separator in '2stems' mode.
separator = Separator('spleeter:2stems')

# Perform the separation.
separator.separate_to_file(INPUT_AUDIO_FOLDER + "/" + INPUT_AUDIO_FILENAME + ".mp3", OUTPUT_AUDIO_FOLDER)


In [7]:
# Load the audio file and extract a feature
outputVocals = OUTPUT_AUDIO_FOLDER + "/" + INPUT_AUDIO_FILENAME + "/" + "vocals" + ".wav"
inputVocals = INPUT_AUDIO_FOLDER + "/" + INPUT_AUDIO_FILENAME + ".mp3"

y, sr = librosa.load(inputVocals, sr=None)
feature = librosa.feature.rms(y=y)

# Use ruptures to detect change points
algo = rpt.Pelt(model="rbf").fit(feature[0])
result = algo.predict(pen=10)

# Visualize the detected change points on the feature
plt.figure(figsize=(10, 6))
rpt.display(feature[0], [], result)
plt.title("Change Point Detection on Audio Feature")
plt.show()

/Users/daryldd/Documents/misc/mmd-generator/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/daryldd/Documents/misc/mmd-generator/.venv/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


BadSegmentationParameters: 

In [ ]:
# Generate a silent track for the length of loaded audio
hop_length = 512

silent_track = np.zeros_like(y)

# Load a beep sound (assumed to be short and sampled at the same rate as the main audio)
beep, _ = librosa.load('beep_test.wav', sr=sr)

# Add a beep sound at detected feature locations
for change_point in result:
  start_sample = change_point * hop_length  # hop_length is the hop size used in feature extraction (default is 512 for RMS)
  end_sample = start_sample + len(beep)
  if end_sample < len(silent_track):
    silent_track[start_sample:end_sample] += beep

# Save the resultant audio
sf.write(OUTPUT_AUDIO_FOLDER + "/" + 'output_with_beeps.wav', silent_track, sr)

In [ ]:
print(result[:10])
